In [1]:
import sympy as sy
import numpy as np
from IPython.display import display
PPTY_PHY_PARAMS = {"positive": True, "real" : True }
PPTY_STATE_VAR  = {"real" : True }

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Objectif
Faire le changement de variable pour un certain nombre de tronçons $N$

In [2]:
N = 5  # Nombre de tronçons
N_lambda = N-1 # Nombre de contraintes
Nx = N * 5     # Nombre total d'état
Nx

25

### Définition des symboles

In [3]:
nuL_vec  = sy.symbols('nu_L1:{}'.format(N+1), **PPTY_STATE_VAR)
nuR_vec  = sy.symbols('nu_R1:{}'.format(N+1), **PPTY_STATE_VAR)
Pi_y_vec = sy.symbols('Pi_y1:{}'.format(N+1), **PPTY_STATE_VAR)
m_vec    = sy.symbols('m_1:{}'.format(N+1), **PPTY_PHY_PARAMS)
h_vec    = sy.symbols('h_1:{}'.format(N+1), **PPTY_STATE_VAR)

In [4]:
X = [0 for i in range(N*2)]
for i in range(0,len(nuL_vec)):
    X[2*i] = nuL_vec[i]
    X[2*i+1] = nuR_vec[i]
    
X = sy.Matrix(X + list(Pi_y_vec)  + list(h_vec)+ list(m_vec))
display(X)

Matrix([
[nu_L1],
[nu_R1],
[nu_L2],
[nu_R2],
[nu_L3],
[nu_R3],
[nu_L4],
[nu_R4],
[nu_L5],
[nu_R5],
[Pi_y1],
[Pi_y2],
[Pi_y3],
[Pi_y4],
[Pi_y5],
[  h_1],
[  h_2],
[  h_3],
[  h_4],
[  h_5],
[  m_1],
[  m_2],
[  m_3],
[  m_4],
[  m_5]])

In [5]:
bT = sy.zeros(N_lambda, Nx)
for i in range(N_lambda):
    bT[i, 1+2*i] = 1 
    bT[i, 2+i*2] = -1
b = bT.T
display(b.T)
print('---------')
print('Contraintes')
display(b.T*X)

Matrix([
[0, 1, -1, 0,  0, 0,  0, 0,  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0,  0, 1, -1, 0,  0, 0,  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0,  0, 0,  0, 1, -1, 0,  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0,  0, 0,  0, 0,  0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

---------
Contraintes


Matrix([
[-nu_L2 + nu_R1],
[-nu_L3 + nu_R2],
[-nu_L4 + nu_R3],
[-nu_L5 + nu_R4]])

In [6]:
# On construit la matrice annulatrice de b à gauche
annul_b = sy.zeros(Nx-N_lambda,Nx)

annul_b[0,0] = 1
annul_b[N::, N + N_lambda::] = sy.eye(3*N+1)
for i in range(1,N_lambda+1):
    annul_b[i, 2*i-1] = 1 
    annul_b[i, 2*i] = 1

display(annul_b, annul_b*b)
print('Rang: ' + str(annul_b.rank()))

assert annul_b.rank() == annul_b.shape[0], "La matrice n'est pas de rang plein"
assert annul_b*b == sy.zeros(Nx-N_lambda, N_lambda), "N'est pas annulateur"

#print(sy.latex(annul_b))

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Rang: 21


## Changement de variable

In [7]:
M = sy.zeros(Nx, Nx)
M[0:Nx-N_lambda,::] = annul_b
M[Nx-N_lambda::, ::] = (bT*b).inv()*bT
display(M)
#print(sy.latex(M))

Matrix([
[1,   0,    0,   0,    0,   0,    0,   0,    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   1,    1,   0,    0,   0,    0,   0,    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   1,    1,   0,    0,   0,    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   1,    1,   0,    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   1,    1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   0,    0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   0,    0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   0,    0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   0,    0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,    0,   0,    0,   0,    0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,   0,    0,   0,  

## Matrice d'interconnexion

Jxx = sy.zeros(Nx, Nx)
Jvw_h = sy.eye(N)

Jql_m = sy.zeros(N, 2*N)
for i in range(N):
    Jql_m[i, 2*i  ] = 1
    Jql_m[i, 2*i+1] = -1
Jxx[4*N::, 0:2*N] = Jql_m
Jxx[3*N:4*N, 2*N:3*N] = Jvw_h
Jxx += Jxx.T
Jxx

R = -Jxx[3*N:5*N, ::].T
R

## Nouveaux états

In [8]:
display(M*X)
#print(sy.latex(M*X))

Matrix([
[             nu_L1],
[     nu_L2 + nu_R1],
[     nu_L3 + nu_R2],
[     nu_L4 + nu_R3],
[     nu_L5 + nu_R4],
[             nu_R5],
[             Pi_y1],
[             Pi_y2],
[             Pi_y3],
[             Pi_y4],
[             Pi_y5],
[               h_1],
[               h_2],
[               h_3],
[               h_4],
[               h_5],
[               m_1],
[               m_2],
[               m_3],
[               m_4],
[               m_5],
[-nu_L2/2 + nu_R1/2],
[-nu_L3/2 + nu_R2/2],
[-nu_L4/2 + nu_R3/2],
[-nu_L5/2 + nu_R4/2]])

In [9]:
bT*M.T

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

## Énergie

In [10]:
nu_NNpa_vec = []
Z = M*X
for i in range(N_lambda):
    Z[i+1] = sy.symbols('nu_{0}{1}'.format(i+1,i+2), **PPTY_STATE_VAR)
for i in range(N_lambda):
    Z[Nx-N_lambda+i] = sy.symbols('Delta_{}'.format(10*(i+1) + (i+2)))
display(Z, M.inv()*Z)
display(M.inv())

Matrix([
[   nu_L1],
[   nu_12],
[   nu_23],
[   nu_34],
[   nu_45],
[   nu_R5],
[   Pi_y1],
[   Pi_y2],
[   Pi_y3],
[   Pi_y4],
[   Pi_y5],
[     h_1],
[     h_2],
[     h_3],
[     h_4],
[     h_5],
[     m_1],
[     m_2],
[     m_3],
[     m_4],
[     m_5],
[Delta_12],
[Delta_23],
[Delta_34],
[Delta_45]])

Matrix([
[              nu_L1],
[ Delta_12 + nu_12/2],
[-Delta_12 + nu_12/2],
[ Delta_23 + nu_23/2],
[-Delta_23 + nu_23/2],
[ Delta_34 + nu_34/2],
[-Delta_34 + nu_34/2],
[ Delta_45 + nu_45/2],
[-Delta_45 + nu_45/2],
[              nu_R5],
[              Pi_y1],
[              Pi_y2],
[              Pi_y3],
[              Pi_y4],
[              Pi_y5],
[                h_1],
[                h_2],
[                h_3],
[                h_4],
[                h_5],
[                m_1],
[                m_2],
[                m_3],
[                m_4],
[                m_5]])

Matrix([
[1,   0,   0,   0,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0],
[0, 1/2,   0,   0,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  0,  0,  0],
[0, 1/2,   0,   0,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,  0,  0,  0],
[0,   0, 1/2,   0,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  1,  0,  0],
[0,   0, 1/2,   0,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, -1,  0,  0],
[0,   0,   0, 1/2,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  1,  0],
[0,   0,   0, 1/2,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0, -1,  0],
[0,   0,   0,   0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  1],
[0,   0,   0,   0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0, -1],
[0,   0,   0,   0,   0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0],
[0,   0,   0,   0,   0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0],
[0,   0,   0

In [11]:
def Ham_inertiel(X):
    Nt = int(len(X)/5)
    Hi = 0
    for i in range(Nt):
        l0 = sy.symbols('ell_{}'.format(i+1), **PPTY_PHY_PARAMS)
        subX = [X[2*i],X[2*i+1],X[2*Nt+i], X[3*Nt+i],X[4*Nt+i]]
        #display(subX)
        Hi += sy.Rational(1,2)*subX[-1]/l0**2*(subX[0]**2 + subX[1]**2 - subX[0]*subX[1])
    return Hi

print("-----------------")
print("Énergie classique")
display(Ham_inertiel(X))
print("-----------------")
print("Énergie contrainte")
display(Ham_inertiel(M.inv()*Z).expand().collect([*m_vec]))
#print(sy.latex(Ham_inertiel(M.inv()*Z).expand().collect([*m_vec])))
print("-----------------")
print("Énergie contrainte développée")
display(Ham_inertiel(M.inv()*Z))
print("Equations algébriques à résoudre")
Z2 = sy.Matrix(Z[Nx-N_lambda::])
display(Ham_inertiel(M.inv()*Z).diff(Z2))

-----------------
Énergie classique


m_5*(nu_L5**2 - nu_L5*nu_R5 + nu_R5**2)/(2*ell_5**2) + m_4*(nu_L4**2 - nu_L4*nu_R4 + nu_R4**2)/(2*ell_4**2) + m_3*(nu_L3**2 - nu_L3*nu_R3 + nu_R3**2)/(2*ell_3**2) + m_2*(nu_L2**2 - nu_L2*nu_R2 + nu_R2**2)/(2*ell_2**2) + m_1*(nu_L1**2 - nu_L1*nu_R1 + nu_R1**2)/(2*ell_1**2)

-----------------
Énergie contrainte


m_1*(Delta_12**2/(2*ell_1**2) + Delta_12*nu_12/(2*ell_1**2) - Delta_12*nu_L1/(2*ell_1**2) + nu_12**2/(8*ell_1**2) - nu_12*nu_L1/(4*ell_1**2) + nu_L1**2/(2*ell_1**2)) + m_2*(Delta_12**2/(2*ell_2**2) + Delta_12*Delta_23/(2*ell_2**2) - Delta_12*nu_12/(2*ell_2**2) + Delta_12*nu_23/(4*ell_2**2) + Delta_23**2/(2*ell_2**2) - Delta_23*nu_12/(4*ell_2**2) + Delta_23*nu_23/(2*ell_2**2) + nu_12**2/(8*ell_2**2) - nu_12*nu_23/(8*ell_2**2) + nu_23**2/(8*ell_2**2)) + m_3*(Delta_23**2/(2*ell_3**2) + Delta_23*Delta_34/(2*ell_3**2) - Delta_23*nu_23/(2*ell_3**2) + Delta_23*nu_34/(4*ell_3**2) + Delta_34**2/(2*ell_3**2) - Delta_34*nu_23/(4*ell_3**2) + Delta_34*nu_34/(2*ell_3**2) + nu_23**2/(8*ell_3**2) - nu_23*nu_34/(8*ell_3**2) + nu_34**2/(8*ell_3**2)) + m_4*(Delta_34**2/(2*ell_4**2) + Delta_34*Delta_45/(2*ell_4**2) - Delta_34*nu_34/(2*ell_4**2) + Delta_34*nu_45/(4*ell_4**2) + Delta_45**2/(2*ell_4**2) - Delta_45*nu_34/(4*ell_4**2) + Delta_45*nu_45/(2*ell_4**2) + nu_34**2/(8*ell_4**2) - nu_34*nu_45/(8*ell_4

-----------------
Énergie contrainte développée


m_5*(nu_R5**2 - nu_R5*(-Delta_45 + nu_45/2) + (-Delta_45 + nu_45/2)**2)/(2*ell_5**2) + m_4*((-Delta_34 + nu_34/2)**2 - (-Delta_34 + nu_34/2)*(Delta_45 + nu_45/2) + (Delta_45 + nu_45/2)**2)/(2*ell_4**2) + m_3*((-Delta_23 + nu_23/2)**2 - (-Delta_23 + nu_23/2)*(Delta_34 + nu_34/2) + (Delta_34 + nu_34/2)**2)/(2*ell_3**2) + m_2*((-Delta_12 + nu_12/2)**2 - (-Delta_12 + nu_12/2)*(Delta_23 + nu_23/2) + (Delta_23 + nu_23/2)**2)/(2*ell_2**2) + m_1*(nu_L1**2 - nu_L1*(Delta_12 + nu_12/2) + (Delta_12 + nu_12/2)**2)/(2*ell_1**2)

Equations algébriques à résoudre


Matrix([
[             m_2*(2*Delta_12 + Delta_23 - nu_12 + nu_23/2)/(2*ell_2**2) + m_1*(2*Delta_12 + nu_12 - nu_L1)/(2*ell_1**2)],
[m_3*(2*Delta_23 + Delta_34 - nu_23 + nu_34/2)/(2*ell_3**2) + m_2*(Delta_12 + 2*Delta_23 - nu_12/2 + nu_23)/(2*ell_2**2)],
[m_4*(2*Delta_34 + Delta_45 - nu_34 + nu_45/2)/(2*ell_4**2) + m_3*(Delta_23 + 2*Delta_34 - nu_23/2 + nu_34)/(2*ell_3**2)],
[             m_5*(2*Delta_45 - nu_45 + nu_R5)/(2*ell_5**2) + m_4*(Delta_34 + 2*Delta_45 - nu_34/2 + nu_45)/(2*ell_4**2)]])

## Energie inertielle en expression matricielle

In [12]:
Xi = sy.Matrix(X[0:2*N])

print("Vecteur des états inertiels: écoulement axial")
display(Xi)
Q_tot = sy.zeros(Nx)

for i in range(N):
    l0 = sy.symbols('ell_{}'.format(i+1), **PPTY_PHY_PARAMS)
    Qi = m_vec[i]/(l0**2)*sy.Matrix([[1,sy.Rational(-1,2)],[sy.Rational(-1,2),1]])
    Q_tot[2*i:2*i+2, 2*i:2*i+2] = Qi

# Matrice de l'énergie contrainte
Q_c = M.inv().T*Q_tot*M.inv()

print("-----------------")
print("Matrice énergie inertielle")
display(Q_tot)
print("-----------------")
print("Matrice énergie inertielle contrainte")
display(Q_c)
print('-----------------')
print("Energie calculée")
display((Z.T*Q_c*Z)[0].expand().collect([*m_vec]))

Vecteur des états inertiels: écoulement axial


Matrix([
[nu_L1],
[nu_R1],
[nu_L2],
[nu_R2],
[nu_L3],
[nu_R3],
[nu_L4],
[nu_R4],
[nu_L5],
[nu_R5]])

-----------------
Matrice énergie inertielle


Matrix([
[     m_1/ell_1**2, -m_1/(2*ell_1**2),                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-m_1/(2*ell_1**2),      m_1/ell_1**2,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[                0,                 0,      m_2/ell_2**2, -m_2/(2*ell_2**2),                 0,                 0,                 0,                 0,                 0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[                0,                 0, -m_2/(2*ell_2**2),      m_2/ell_2**2,                 0,                 0,                 0,                 0,                 0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[                0,                 0,     

-----------------
Matrice énergie inertielle contrainte


Matrix([
[     m_1/ell_1**2,                    -m_1/(4*ell_1**2),                                    0,                                    0,                                    0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                    -m_1/(2*ell_1**2),                                    0,                                    0,                                    0],
[-m_1/(4*ell_1**2),  m_2/(4*ell_2**2) + m_1/(4*ell_1**2),                    -m_2/(8*ell_2**2),                                    0,                                    0,                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -m_2/(2*ell_2**2) + m_1/(2*ell_1**2),                    -m_2/(4*ell_2**2),                                    0,                                    0],
[                0,                    -m_2/(8*ell_2**2),  m_3/(4*ell_3**2) + m_2/(4*ell_2**2),                    -m_3/(8*ell_3**2),                                    0,                 0, 0, 0, 0, 0, 0, 0, 0, 0

-----------------
Energie calculée


m_1*(Delta_12**2/ell_1**2 + Delta_12*nu_12/ell_1**2 - Delta_12*nu_L1/ell_1**2 + nu_12**2/(4*ell_1**2) - nu_12*nu_L1/(2*ell_1**2) + nu_L1**2/ell_1**2) + m_2*(Delta_12**2/ell_2**2 + Delta_12*Delta_23/ell_2**2 - Delta_12*nu_12/ell_2**2 + Delta_12*nu_23/(2*ell_2**2) + Delta_23**2/ell_2**2 - Delta_23*nu_12/(2*ell_2**2) + Delta_23*nu_23/ell_2**2 + nu_12**2/(4*ell_2**2) - nu_12*nu_23/(4*ell_2**2) + nu_23**2/(4*ell_2**2)) + m_3*(Delta_23**2/ell_3**2 + Delta_23*Delta_34/ell_3**2 - Delta_23*nu_23/ell_3**2 + Delta_23*nu_34/(2*ell_3**2) + Delta_34**2/ell_3**2 - Delta_34*nu_23/(2*ell_3**2) + Delta_34*nu_34/ell_3**2 + nu_23**2/(4*ell_3**2) - nu_23*nu_34/(4*ell_3**2) + nu_34**2/(4*ell_3**2)) + m_4*(Delta_34**2/ell_4**2 + Delta_34*Delta_45/ell_4**2 - Delta_34*nu_34/ell_4**2 + Delta_34*nu_45/(2*ell_4**2) + Delta_45**2/ell_4**2 - Delta_45*nu_34/(2*ell_4**2) + Delta_45*nu_45/ell_4**2 + nu_34**2/(4*ell_4**2) - nu_34*nu_45/(4*ell_4**2) + nu_45**2/(4*ell_4**2)) + m_5*(Delta_45**2/ell_5**2 - Delta_45*nu_45/e

Les $N_\lambda$ dernières lignes sont celles qui nous intéressent puisque'elles donnent le gradient en fonction des variables $\Delta$. Ces lignes peuvent être décomposées en deux matrices: 
   - le bloc tout en bas à gauche, que l'on nommera $Q_{12}^\intercal$
   - la matrice carrée en bas à droite, que l'on va inverser (mon dieu, ne serait-ce pas des masses équivalentes?) que l'on nommera $Q_{22}$

In [13]:
Q12 = Q_c[Nx-N_lambda::,0:N+1]
Q22 = Q_c[Nx-N_lambda::, Nx-N_lambda::]
display(Q12,Q22)

Matrix([
[-m_1/(2*ell_1**2), -m_2/(2*ell_2**2) + m_1/(2*ell_1**2),                     m_2/(4*ell_2**2),                                    0,                                    0,                0],
[                0,                    -m_2/(4*ell_2**2), -m_3/(2*ell_3**2) + m_2/(2*ell_2**2),                     m_3/(4*ell_3**2),                                    0,                0],
[                0,                                    0,                    -m_3/(4*ell_3**2), -m_4/(2*ell_4**2) + m_3/(2*ell_3**2),                     m_4/(4*ell_4**2),                0],
[                0,                                    0,                                    0,                    -m_4/(4*ell_4**2), -m_5/(2*ell_5**2) + m_4/(2*ell_4**2), m_5/(2*ell_5**2)]])

Matrix([
[m_2/ell_2**2 + m_1/ell_1**2,            m_2/(2*ell_2**2),                           0,                           0],
[           m_2/(2*ell_2**2), m_3/ell_3**2 + m_2/ell_2**2,            m_3/(2*ell_3**2),                           0],
[                          0,            m_3/(2*ell_3**2), m_4/ell_4**2 + m_3/ell_3**2,            m_4/(2*ell_4**2)],
[                          0,                           0,            m_4/(2*ell_4**2), m_5/ell_5**2 + m_4/ell_4**2]])

In [14]:
#Q22inv = Q22.inv()
f = lambda mat : mat.simplify()

In [15]:
#Q22inv

Au passage, $Q_{12}$ peut être décomposé en une partie anti symétrique et une partie diagonale.

## Manipulations
Inverser directement la matrice Q22 est envisageable pour 2 tronçons, ou 3. Cependant, la matrice est complètement pleine et on est vite coincé.

On va donc tenté de simplifier le problème, notamment en utilisant des méthodes adaptées.

### Décomposition LU
$PA = LU$ avec:
    1. $L$ une matrice triangulaire inférieure avec identité à la diagonale
    2. $U$ est triangulaire supérieure
    3. $P$ est "row swap index pairs"

In [16]:
L,U, Perm = Q22.LUdecomposition()

In [17]:
display(L, U)

Matrix([
[                                             1,                                                                                                0,                                                                                                                                                                   0, 0],
[                                             0,                                                                                                1,                                                                                                                                                                   0, 0],
[                                             0,                                                                                                0,                                                                                                                                                                   1, 0],
[m_2/(2*ell_2**2*(m_2/ell_2**2 + m_1/ell_1*

Matrix([
[m_2/ell_2**2 + m_1/ell_1**2, m_2/(2*ell_2**2),                           0,                                                                                                                                                                                                                                                                                                                  0],
[                          0, m_3/(2*ell_3**2), m_4/ell_4**2 + m_3/ell_3**2,                                                                                                                                                                                                                                                                                                   m_4/(2*ell_4**2)],
[                          0,                0,            m_4/(2*ell_4**2),                                                                                                                                               

## Décomposition QR
$A = Q*R$ où $Q$ est orthogonale et $R$ est triangulaire supérieure.

ERRATUM: c'est moche

## Cholesky
Décomposition $A = L L^\intercal$

In [18]:
L = Q22.cholesky(hermitian=False)

In [19]:
display(L)

Matrix([
[                 sqrt(m_2/ell_2**2 + m_1/ell_1**2),                                                                                                      0,                                                                                                                                                          0,                                                                                                                                                                                             0],
[m_2/(2*ell_2**2*sqrt(m_2/ell_2**2 + m_1/ell_1**2)),                  sqrt(m_3/ell_3**2 + m_2/ell_2**2 - m_2**2/(4*ell_2**4*(m_2/ell_2**2 + m_1/ell_1**2))),                                                                                                                                                          0,                                                                                                                                                                              

## LDL
Variante de Cholesky: $A=LDL^\intercal$, symmetric indefinite facorization.

In [20]:
L, D = Q22.LDLdecomposition(hermitian=False)

In [21]:
display(L,D)

Matrix([
[                                             1,                                                                                                  0,                                                                                                                                                      0, 0],
[m_2/(2*ell_2**2*(m_2/ell_2**2 + m_1/ell_1**2)),                                                                                                  1,                                                                                                                                                      0, 0],
[                                             0, m_3/(2*ell_3**2*(m_3/ell_3**2 + m_2/ell_2**2 - m_2**2/(4*ell_2**4*(m_2/ell_2**2 + m_1/ell_1**2)))),                                                                                                                                                      1, 0],
[                                             0,                            

Matrix([
[m_2/ell_2**2 + m_1/ell_1**2,                                                                               0,                                                                                                                                   0,                                                                                                                                                                                       0],
[                          0, m_3/ell_3**2 + m_2/ell_2**2 - m_2**2/(4*ell_2**4*(m_2/ell_2**2 + m_1/ell_1**2)),                                                                                                                                   0,                                                                                                                                                                                       0],
[                          0,                                                                               0, m_4/ell_4**2 + m_3/e

Question: Est-ce que $D$ est inversible?

In [22]:
D.det().simplify()

m_2*m_3*m_4*m_5/(ell_2**2*ell_3**2*ell_4**2*ell_5**2) + 3*m_2*m_3*m_4**2/(4*ell_2**2*ell_3**2*ell_4**4) + 3*m_2*m_3**2*m_5/(4*ell_2**2*ell_3**4*ell_5**2) + 3*m_2*m_3**2*m_4/(4*ell_2**2*ell_3**4*ell_4**2) + 3*m_2**2*m_4*m_5/(4*ell_2**4*ell_4**2*ell_5**2) + 9*m_2**2*m_4**2/(16*ell_2**4*ell_4**4) + 3*m_2**2*m_3*m_5/(4*ell_2**4*ell_3**2*ell_5**2) + 3*m_2**2*m_3*m_4/(4*ell_2**4*ell_3**2*ell_4**2) + m_1*m_3*m_4*m_5/(ell_1**2*ell_3**2*ell_4**2*ell_5**2) + 3*m_1*m_3*m_4**2/(4*ell_1**2*ell_3**2*ell_4**4) + 3*m_1*m_3**2*m_5/(4*ell_1**2*ell_3**4*ell_5**2) + 3*m_1*m_3**2*m_4/(4*ell_1**2*ell_3**4*ell_4**2) + m_1*m_2*m_4*m_5/(ell_1**2*ell_2**2*ell_4**2*ell_5**2) + 3*m_1*m_2*m_4**2/(4*ell_1**2*ell_2**2*ell_4**4) + m_1*m_2*m_3*m_5/(ell_1**2*ell_2**2*ell_3**2*ell_5**2) + m_1*m_2*m_3*m_4/(ell_1**2*ell_2**2*ell_3**2*ell_4**2)

Réponse: oui, car son déterminant est toujours positif

In [23]:
L.det().simplify()

1

## Inverser
$D$ est diagonale et noe pose donc aucun problème. $L$ est bi-diagonale, faut chercher.

In [24]:
L

Matrix([
[                                             1,                                                                                                  0,                                                                                                                                                      0, 0],
[m_2/(2*ell_2**2*(m_2/ell_2**2 + m_1/ell_1**2)),                                                                                                  1,                                                                                                                                                      0, 0],
[                                             0, m_3/(2*ell_3**2*(m_3/ell_3**2 + m_2/ell_2**2 - m_2**2/(4*ell_2**4*(m_2/ell_2**2 + m_1/ell_1**2)))),                                                                                                                                                      1, 0],
[                                             0,                            

In [26]:
Linv = L.inv()

In [29]:
Linv

Matrix([
[                                                                                                                                                                                                                                                                                                1,                                                                                                                                                                                                                                                     0,                                                                                                                                                       0, 0],
[                                                                                                                                                                                                                                                  -m_2/(2*ell_2**2*(m_2/ell_2**2 + m_1/ell_1**2)),    

In [31]:
LinvDinvLinvT = Linv*D.inv()*Linv.T

In [32]:
LinvDinvLinvT.applyfunc(f)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                      ell_1**2*ell_2**2/(ell_1**2*m_2 + ell_2**2*m_1),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        